In [1]:
import requests
import lxml.html as lx
import re
import pandas as pd
import time
import matplotlib as plt

In [7]:
import requests
import lxml.html as lx
base_link = 'https://tracker.gg'
challengers_prof = []
for i in range(2):
    response = requests.get(f'https://tracker.gg/lol/leaderboards/stats/all/LeaguePoints?region=NA&queueType=RANKED_SOLO_5x5&page={i}')
    page = lx.fromstring(response.text)
    curr_challengers = page.xpath('//*[@id="app"]/div[2]/div[3]/div/main/div[3]/div[2]/div/div/div[1]/div/table/tbody/tr//div[contains(@class, "text")]/a/@href')
    for link in curr_challengers:
        if link.startswith('/lol/profile'):
            challengers_prof.append(base_link + link)

In [30]:
len(challengers_prof)

99

In [34]:
players_df = pd.DataFrame(columns=['Position', 'Matches Played', 'K/D/A', 'Win Rate', 'Kill Participation', 'Objective Participation'])

In [10]:
for prof in challengers_prof:
    response = requests.get(prof)
    page = lx.fromstring(response.text)
    try:
        top_position = page.xpath('//*[@id="app"]/div[2]/div[3]/div/main/div[3]/div[3]/div[2]/div[1]/div[2]/div/div[1]/div[2]/div[1]/text()')[0]
        matches_played = page.xpath('//*[@id="app"]/div[2]/div[3]/div/main/div[3]/div[3]/div[2]/div[2]/div[1]/div[1]/div/div/div/div/span[2]/text()')[0].strip()
        matches_played = int(re.sub(r'\D', '', matches_played))
        kda = float(page.xpath('//*[@id="app"]/div[2]/div[3]/div/main/div[3]/div[3]/div[2]/div[2]/div[1]/div[3]/div[1]/div/div[2]/span[2]/span/text()')[0])
        winrate = float(page.xpath('//*[@id="app"]/div[2]/div[3]/div/main/div[3]/div[3]/div[2]/div[2]/div[1]/div[2]/div/div/div[1]/div[2]/span[2]/text()')[0].replace('%', ''))
        kill_participation = float(page.xpath('//*[@id="app"]/div[2]/div[3]/div/main/div[3]/div[3]/div[2]/div[2]/div[2]/div[2]/div[2]/div[2]/div[2]/text()')[0].replace('%', ''))
        objective_participation = float(page.xpath('//*[@id="app"]/div[2]/div[3]/div/main/div[3]/div[3]/div[2]/div[2]/div[2]/div[2]/div[2]/div[3]/div[2]/text()')[0].replace('%', ''))
        curr_player = {'Position': top_position, 'Matches Played': matches_played, 'K/D/A': kda, 'Win Rate': winrate, 'Kill Participation': kill_participation, 'Objective Participation': objective_participation}
        curr_player_df = pd.DataFrame([curr_player], columns=['Position', 'Matches Played', 'K/D/A', 'Win Rate', 'Kill Participation', 'Objective Participation'])
        players_df = pd.concat([players_df, curr_player_df], ignore_index=True)
    except Exception as e:
        print(prof)


In [3]:
players_df = pd.read_csv('challenger_players_df.csv')
players_df = players_df.drop('Unnamed: 0', axis=1)
players_df

,Position,Matches Played,K/D/A,Win Rate,Kill Participation,Objective Participation
0,Middle,247,3.65,66.8,54.1,49.3
1,Jungle,191,4.19,69.6,62.4,76.9
2,Jungle,516,2.90,55.6,59.9,72.9
3,Top,593,2.64,54.3,42.0,42.6
4,Jungle,356,3.59,56.2,63.0,74.7
...,...,...,...,...,...,...
95,Bottom,275,3.33,55.3,48.8,48.4
96,Middle,233,1.90,57.5,47.9,32.5
97,Support,356,3.24,53.4,57.8,30.1
98,Support,274,4.07,59.1,64.0,54.6


In [15]:
players_df.to_csv('challenger_players_df.csv') #export so don't have to run code everytime git pull

In [91]:
riot_api = 'RGAPI-285344e4-f5a0-4baa-82ac-6fe59e537a4e'

In [92]:
link = 'https://na1.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/PLATINUM/IV?page=1&api_key=RGAPI-285344e4-f5a0-4baa-82ac-6fe59e537a4e'
response = requests.get(link, headers={
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com"
})

In [93]:
plat_summoners = response.json()
plat_summoners

[{'leagueId': 'a8d9bcb1-1df5-4743-a197-8a33249b052a',
  'queueType': 'RANKED_SOLO_5x5',
  'tier': 'PLATINUM',
  'rank': 'IV',
  'summonerId': 'qXSjj3H1P-ixuaSND2f23qsIKsu9qnFlh4y0Yl0udaUx2jKP',
  'summonerName': 'big grammy',
  'leaguePoints': 0,
  'wins': 35,
  'losses': 23,
  'veteran': False,
  'inactive': False,
  'freshBlood': False,
  'hotStreak': False},
 {'leagueId': 'f885eea3-e3c8-4dad-8cc1-ed3b524b66cc',
  'queueType': 'RANKED_SOLO_5x5',
  'tier': 'PLATINUM',
  'rank': 'IV',
  'summonerId': 'ni8oRtiJPZv71mkfzemrCWSmqFG7iN4cYAiRXRZcokYKksgQ',
  'summonerName': 'WhoAmIIAmWho',
  'leaguePoints': 75,
  'wins': 43,
  'losses': 42,
  'veteran': False,
  'inactive': False,
  'freshBlood': False,
  'hotStreak': False},
 {'leagueId': '83146289-4e30-4715-b353-9e42101f558a',
  'queueType': 'RANKED_SOLO_5x5',
  'tier': 'PLATINUM',
  'rank': 'IV',
  'summonerId': 'xRB5poO_mAkbfdV_c2sqTJf89XQnmEnBVBjPciqkrIppquey',
  'summonerName': 'AHS J1mmy',
  'leaguePoints': 43,
  'wins': 42,
  'losse

In [94]:
plat_summoners = response.json()
plat_summoner_names = {}
for summoner in plat_summoners[:200]:
    summoner_name = summoner['summonerName'].replace(' ', '%20')
    plat_summoner_names[summoner_name] = 0
plat_summoner_names

{'big%20grammy': 0,
 'WhoAmIIAmWho': 0,
 'AHS%20J1mmy': 0,
 'jeonyeobeen': 0,
 'K1llerTaco666': 0,
 'sewer%20peepee': 0,
 'floofers': 0,
 'lMarvisl': 0,
 'BaneOfTheEndersX': 0,
 'Icelonk1': 0,
 '20cm%20of%20love20': 0,
 'Les%20Marukoko': 0,
 'Lnotstylish99': 0,
 'shiling': 0,
 'SharkNaDan': 0,
 'ElementNe': 0,
 'ArdentlyToxic': 0,
 'THE%20CÜM%20LORD': 0,
 'thiev3ss': 0,
 'thatyasuootp': 0,
 'xDzero': 0,
 'KILL3R%20FOR3V3R': 0,
 'i%20eated%20a%20fork': 0,
 'h1de%20in%20brush': 0,
 'slapdady': 0,
 'Darkmage9876': 0,
 'Wettcomerag': 0,
 'Dashertheruler': 0,
 'deafening%20voices': 0,
 'Want%20U%20Stay': 0,
 'emanthegreates': 0,
 'SkieTales': 0,
 'SharkNinja8': 0,
 'Terrien': 0,
 '181008': 0,
 'ChumOnTheWave': 0,
 'Taota0': 0,
 'As9223': 0,
 'idiothead69': 0,
 'KindKO': 0,
 'ReisenUdongInaba': 0,
 'QiuQiuRen': 0,
 'ReapFish': 0,
 'Van2%20NA': 0,
 'lyingmando322': 0,
 'Cold8': 0,
 'ThomasJohnMarvin': 0,
 'MASTERPO0PER': 0,
 'clown%20in%20town': 0,
 'Du%20Çouteau': 0,
 'Niccgutz': 0,
 'Buckar

In [101]:
for summoner in list(plat_summoner_names.keys())[100:200]:
    link = f'https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/{summoner}'
    response = requests.get(link, headers={
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com",
    "X-Riot-Token": "RGAPI-285344e4-f5a0-4baa-82ac-6fe59e537a4e"
    })
    response = response.json()
    plat_summoner_names[summoner] = response['puuid']
plat_summoner_names

{'big%20grammy': 'FNSKHpNE9nYP53YnYJGcIHszRPA5HtAMbDsLIccGyb3yjvBHPmarINBG7CCEJv_l9mNIoArPzZJdZA',
 'WhoAmIIAmWho': 'jgXPB6YtSrNLeBGZhH7oespSLWeVp19YPANJiIulKkXc_joPi3h8tUzjx5LytZ3LWRNza6uDTwhgQw',
 'AHS%20J1mmy': '4hNJaIolYANz6XCM127izS2KXmBRb9PJEfM5n7i8U6sW2K5U9uPAlnLJWOYNFIh05-Q5VE6MoEv9gQ',
 'jeonyeobeen': 'r4wsu8q6bVOwgONHkgQUiwC05pVh5SRl2OQeExeGGTlWh8FkVyQyyCO9rxyFX4qVdjq8lcT0ZMuNlw',
 'K1llerTaco666': 'iggpgjWzmCsfISf1sF_aeDaSw0GP_A-r0hmgmZZwrzyv45QcwJA2jIekQJ2H278Jc8W7qMmt7tkPmA',
 'sewer%20peepee': 'e0aeLH-9fjxvxFJrk2-znuNO0klD4zX8hG0F7WfrAuozMuxVXr71acSRzwaNugn1npWdfb-vOI3bDA',
 'floofers': 'l8FsC4_ks0pWsvC5aGELCXpIDCvA_DDQFajSfSWhh-9bBkMdoxuxanlsTVtqYR5b2b9dDoyZwrZUQA',
 'lMarvisl': 'W9i7_2IAocoiop98lDLI17VB7HQmDV9eq3SlZ5j9W23Q9G2SQg_-SWqTjeYEaOgQidVzZlF_1DpomA',
 'BaneOfTheEndersX': 'BO4WhKrQnrPKX_XFGqluMa3IjrkQ0oSbLkTJ-XIuWSx7UwFm4-gunCnyEyLctYkWvTazZoMML1zYjA',
 'Icelonk1': 'Eq3HE-Icc4jydDLnL7k6vyD3sJ4a7eWkVAVz_sWJ9lXF5TkHUE3mTLAYirf5EzjlXmKg1O-SxxCCSw',
 '20cm%20of%20lov

In [ ]:
full_id = []

In [117]:
for name, ppuid in list(plat_summoner_names.items())[:100]:
    link = f'https://americas.api.riotgames.com/riot/account/v1/accounts/by-puuid/{ppuid}'
    response = requests.get(link, headers={
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com",
    "X-Riot-Token": "RGAPI-285344e4-f5a0-4baa-82ac-6fe59e537a4e"
    })
    tagline = '%23' + response.json()['tagLine']
    full_id.append(name + tagline)
full_id

['SSBM%20Instinct%23NA1',
 'Mashuaifu%232ccc',
 'cookiemelikey%230410',
 'Mhunch%23NA1',
 'JogOk%23JGY',
 'Just%20For%20One%20Day%23NA1',
 'Sc0ttyTr0ng%23jd21',
 'Nvron%23NA1',
 'AirJuice%23NA1',
 'f1tzyxd%23NA1',
 'JFKennendy%23NA1',
 'Risky%20Investment%23NA1',
 'ian%20bøt%23NA1',
 'Mastercroz%23NA1',
 'BigBoyBrosie%23NA1',
 'Semurei%23NA1',
 'TODDOFVIDEOGAMES%23TOAD',
 'Śeven%23NA1',
 'Captain599%23NA1',
 'Stand%20at%20the%20top%23NA1',
 'PatitosGaming%23NA1',
 'Yodq%23drury',
 'scubadiver%20scrub%23NA1',
 's0ggywafffls%23NA1',
 'Giørno%20Giovanna%234444',
 'Iron5Dive%23NA1',
 'k9s%23NA1',
 'Kraxm14%23BONK',
 'Old%20Tree%20Pan%20Gen%23777',
 'McNugget707%23NA1',
 'JIggy1%233732',
 'Najilis%23NA1',
 'Wölhaiksong%23NA1',
 'Real%20Faker%23peek',
 'xXmouazXx%236175',
 'GriimJoow7%23NA1',
 'nowjoeblessedup%23NA1',
 'VietNømNømNøm%23Munch',
 'Kuroisi%23NA1',
 'GrandRichard%23NA1',
 'lowhp%20gamebegin%23NA1',
 'Salamiiin%23NA1',
 'DeWeebler%23NA1',
 'devilsniper14%23NA1',
 'Cosmo%20Canyon%

In [5]:
plat_players_df = pd.read_csv('plat_players_df.csv')
plat_players_df = plat_players_df.drop('Unnamed: 0', axis=1)

In [6]:
import csv
temp = []
with open('full_id.csv', 'r', encoding='utf-8') as file:
    reader = csv.reader(file)
    for row in reader:
        temp.append(row)
temp_id = []
for lol in temp:
    lol = ','.join(lol)
    lol = lol.replace(',', '')
    temp_id.append(lol)
    full_id = temp_id

In [112]:
plat_players_df = pd.DataFrame(columns=['Position', 'Matches Played', 'K/D/A', 'Win Rate', 'Kill Participation', 'Objective Participation'])

In [9]:
for id in full_id[50:60]:
    link = f'https://tracker.gg/lol/profile/riot/NA/{id}/overview'
    response = requests.get(link)
    page = lx.fromstring(response.text)
    try:
        top_position = page.xpath('//*[@id="app"]/div[2]/div[3]/div/main/div[3]/div[3]/div[2]/div[1]/div[2]/div/div[1]/div[2]/div[1]/text()')[0]
        matches_played = page.xpath('//*[@id="app"]/div[2]/div[3]/div/main/div[3]/div[3]/div[2]/div[2]/div[1]/div[1]/div/div/div/div/span[2]/text()')[0].strip()
        matches_played = int(re.sub(r'\D', '', matches_played))
        kda = float(page.xpath('//*[@id="app"]/div[2]/div[3]/div/main/div[3]/div[3]/div[2]/div[2]/div[1]/div[3]/div[1]/div/div[2]/span[2]/span/text()')[0])
        winrate = float(page.xpath('//*[@id="app"]/div[2]/div[3]/div/main/div[3]/div[3]/div[2]/div[2]/div[1]/div[2]/div/div/div[1]/div[2]/span[2]/text()')[0].replace('%', ''))
        kill_participation = float(page.xpath('//*[@id="app"]/div[2]/div[3]/div/main/div[3]/div[3]/div[2]/div[2]/div[2]/div[2]/div[2]/div[2]/div[2]/text()')[0].replace('%', ''))
        objective_participation = float(page.xpath('//*[@id="app"]/div[2]/div[3]/div/main/div[3]/div[3]/div[2]/div[2]/div[2]/div[2]/div[2]/div[3]/div[2]/text()')[0].replace('%', ''))
        curr_player = {'Position': top_position, 'Matches Played': matches_played, 'K/D/A': kda, 'Win Rate': winrate, 'Kill Participation': kill_participation, 'Objective Participation': objective_participation}
        curr_player_df = pd.DataFrame([curr_player], columns=['Position', 'Matches Played', 'K/D/A', 'Win Rate', 'Kill Participation', 'Objective Participation'])
        plat_players_df = pd.concat([plat_players_df, curr_player_df], ignore_index=True)
    except Exception as e:
        print(id)


NookiiVT%236969
Jamei12%23NA1
iKP%23NA2
Twon09%230909
ł%20Amaze%20Myself%23NA1
SkacaT%23NA1
nolxn%235454
MikoTime%23NA1
Shentist%23NA1
Jzay%20Nightmare%231703


In [8]:
plat_players_df

,Position,Matches Played,K/D/A,Win Rate,Kill Participation,Objective Participation
0,Middle,37,3.31,48.6,47.7,42.4
1,Top,60,1.60,45.0,41.4,39.3
2,Support,12,4.36,41.7,58.8,59.8
3,Top,4,2.79,50.0,54.1,48.0
4,Top,49,1.88,55.1,39.9,53.2
5,Bottom,6,2.05,33.3,40.7,31.1
6,Jungle,38,2.60,55.3,45.7,56.3
7,Top,17,1.67,41.2,41.2,37.4
8,Support,4,6.18,75.0,65.1,80.6
9,Bottom,1,2.20,0.0,64.7,18.8


In [162]:
import csv
plat_players_df.to_csv('plat_players_df.csv')
with open('full_id.csv', mode = 'w') as file:
    writer = csv.writer(file)
    writer.writerows(full_id)

In [163]:
temp = []
with open('full_id.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        temp.append(row)
temp_id = []
for lol in temp:
    lol = ','.join(lol)
    lol = lol.replace(',', '')
    temp_id.append(lol)
temp_id

['SSBM%20Instinct%23NA1',
 'Mashuaifu%232ccc',
 'cookiemelikey%230410',
 'Mhunch%23NA1',
 'JogOk%23JGY',
 'Just%20For%20One%20Day%23NA1',
 'Sc0ttyTr0ng%23jd21',
 'Nvron%23NA1',
 'AirJuice%23NA1',
 'f1tzyxd%23NA1',
 'JFKennendy%23NA1',
 'Risky%20Investment%23NA1',
 'ian%20bøt%23NA1',
 'Mastercroz%23NA1',
 'BigBoyBrosie%23NA1',
 'Semurei%23NA1',
 'TODDOFVIDEOGAMES%23TOAD',
 'Śeven%23NA1',
 'Captain599%23NA1',
 'Stand%20at%20the%20top%23NA1',
 'PatitosGaming%23NA1',
 'Yodq%23drury',
 'scubadiver%20scrub%23NA1',
 's0ggywafffls%23NA1',
 'Giørno%20Giovanna%234444',
 'Iron5Dive%23NA1',
 'k9s%23NA1',
 'Kraxm14%23BONK',
 'Old%20Tree%20Pan%20Gen%23777',
 'McNugget707%23NA1',
 'JIggy1%233732',
 'Najilis%23NA1',
 'Wölhaiksong%23NA1',
 'Real%20Faker%23peek',
 'xXmouazXx%236175',
 'GriimJoow7%23NA1',
 'nowjoeblessedup%23NA1',
 'VietNømNømNøm%23Munch',
 'Kuroisi%23NA1',
 'GrandRichard%23NA1',
 'lowhp%20gamebegin%23NA1',
 'Salamiiin%23NA1',
 'DeWeebler%23NA1',
 'devilsniper14%23NA1',
 'Cosmo%20Canyon%